In [1]:
import os
import sys
import numpy as np

In [67]:
class genAmorphous:
    def __init__(self, 
                 density, 
                 chem_formula,
                 clear_dummy = True):
        """
        Argument 1: (float) density in g/cm3 (ex. 10.00)
        Argument 2: (str) chemical forumula (ex. Hf32O64Ag1)
        Argument 3: (bool, default:True) remove dummy files
        """
        self.density = density
        self.chem_formula = chem_formula

        self.atomMass = {}
        self.read_atomic_mass_table()

        self.atom_name = []
        self.atom_num = []
        self.divide_chem_formula()

        # check atom name
        for name in self.atom_name:
            if not(name in self.atomMass.keys()):
                print(f"{name} is unknown atom.")
                sys.exit(0)

        self.alat = None
        self.alat_pbc = None
        self.get_dimension_cubic()

        self.write_pdb()
        self.write_packmol_input()

        # run packmol
        os.system("packmol < packmol.inp > packmol.out")

        # write POSCAR
        self.covert_pdb_to_poscar()

        if clear_dummy:
            self.clear_dummy_files()

    def read_atomic_mass_table(self):
        if not(os.path.isfile('atomic_mass_table.dat')):
            print('atomic_mass_table.dat is not found.\n \
                  please visit https://github.com/TY-Jeong/arpaca')
            sys.exit(0)
        lines = [line.strip() for line in open('atomic_mass_table.dat')]
        for line in lines:
            [name, mass] = line.split('\t')
            self.atomMass[name] = float(mass)

    def divide_chem_formula(self):
        check, word = 'alpha', ''
        for letter in self.chem_formula:
            if letter.isalpha():
                if check == 'alpha':
                    word += letter
                elif check == 'num':
                    self.atom_num += [int(word)]
                    check = 'alpha'
                    word = letter
            else:
                if check == 'alpha':
                    self.atom_name += [word]
                    check = 'num'
                    word = letter
                elif check == 'num':
                    word += letter
        self.atom_num += [int(word)]
        self.atom_num = np.array(self.atom_num)

    def get_dimension_cubic(self):
        u  =  1.660539e-24 # g
        mass = [self.atomMass[name]*u for name in self.atom_name]
        mass = np.array(mass)
        m = np.sum(mass * self.atom_num)
        self.alat = (m/self.density)**(1/3)*1e8  # Å
        self.alat_pbc = self.alat*0.98 - 2

    def write_pdb(self):
        for name in self.atom_name:
            with open(f"./{name}.pdb",'w') as f:
                f.write(f"COMPND    {name}_atom\n")
                f.write(f"COMPND   1Created by aRPaCa\n")
                f.write("HETATM    1 %-2s           1       5.000   5.000   5.000\n"%(name))
                f.write("END")

    def write_packmol_input(self):
        with open("./packmol.inp",'w') as f:
            f.write("tolerance 2.0\n")
            f.write("filetype pdb\n")
            f.write("output packmol_output.pdb\n\n")

            for name, num in zip(self.atom_name,self.atom_num):
                f.write(f"structure {name}.pdb\n")
                f.write(f"  number {num}\n")
                f.write("  inside cube 0. 0. 0. %.6f \n"%self.alat_pbc)
                f.write("end structure\n\n")

    def covert_pdb_to_poscar(self):
        lines = [line.strip() for line in open('./packmol_output.pdb')]
        num_tot = np.sum(self.atom_num)
        shift = (self.alat-self.alat_pbc)/2
        with open("./POSCAR_"+self.chem_formula, 'w') as f:
            f.write("Generated by aRPaCa\n")
            f.write("1.0\n")
            f.write("   %.6f      0.000000      0.000000\n"%self.alat)
            f.write("   0.000000      %.6f      0.000000\n"%self.alat)
            f.write("   0.000000      0.000000      %.6f\n"%self.alat)
            for name in self.atom_name:
                f.write("   "+name)
            f.write("\n")
            for num in self.atom_num:
                f.write("   "+str(num))
            f.write("\n")
            f.write("Cartesian\n")
            for line in lines[5:5+num_tot]:
                coord = np.array(line.split()[-3:], dtype=float)
                coord += np.array([shift, shift, shift])
                f.write("    %.6f    %.6f   %.6f\n"%(coord[0], coord[1], coord[2]))
    
    def clear_dummy_files(self):
        for name in self.atom_name:
            os.remove(f"{name}.pdb")
        os.remove('packmol.inp')
        os.remove('packmol.out')
        os.remove('packmol_output.pdb')

In [68]:
test = genAmorphous(10.00, 'Hf32O64')